In [116]:
import numpy as np
import pandas as pd

import itertools
import copy

# Define Game-Tree

In [2]:
class GameTree_Kuhn2P:
    def __init__(self):
        self.gameTree = {0: {"name": "START", 
                             "thisPlayer": None, "nextPlayer": "p0",
                             "thisRound": 0, "nextRound": 1,
                             "conn_up": None, "conn_down": []}}
        self.playerActions = {"p0": [],
                              "p1": []}
        
        self.addRound_1()
        self.addRound_2()
        self.addRound_3()
       
    # ----------------------------------------------------------------------------------------------------
    #          first round
    # ----------------------------------------------------------------------------------------------------
    def addRound_1(self):
        thisRound, nextRound = 1, 2
        nodeUp  = 0
        
        nodeNew = 1
        self.gameTree[nodeUp]["conn_down"].append(nodeNew)
        self.gameTree[nodeNew] = {"name": "check", 
                                  "thisPlayer": "p0", "nextPlayer": "p1", 
                                  "thisRound": thisRound, "nextRound": nextRound,
                                  "conn_up": nodeUp, "conn_down": []}
        self.playerActions["p0"].append(nodeNew)
        
        nodeNew = 2
        self.gameTree[nodeUp]["conn_down"].append(nodeNew)
        self.gameTree[nodeNew] = {"name": "bet", 
                                  "thisPlayer": "p0", "nextPlayer": "p1", 
                                  "thisRound": thisRound, "nextRound": nextRound,
                                  "conn_up": nodeUp, "conn_down": []}
        self.playerActions["p0"].append(nodeNew)
        
    # ----------------------------------------------------------------------------------------------------
    #          second round
    # ----------------------------------------------------------------------------------------------------  
    def addRound_2(self):
        thisRound, nextRound = 2, 3
        
        # responses to P1:check
        nodeUp  = 1
        nodeNew = 3
        self.gameTree[nodeUp]["conn_down"].append(nodeNew)
        self.gameTree[nodeNew] = {"name": "check", 
                                  "thisPlayer": "p1", "nextPlayer": None, 
                                  "thisRound": thisRound, "nextRound": None,
                                  "conn_up": nodeUp, "conn_down": [],
                                  "winningPlayer": "SHOWDOWN",
                                  "winAmount": 1}
        self.playerActions["p1"].append(nodeNew)
        
        nodeNew = 4
        self.gameTree[nodeUp]["conn_down"].append(nodeNew)
        self.gameTree[nodeNew] = {"name": "bet", 
                                  "thisPlayer": "p1", "nextPlayer": "p0", 
                                  "thisRound": thisRound, "nextRound": nextRound,
                                  "conn_up": nodeUp, "conn_down": []}
        self.playerActions["p1"].append(nodeNew)
        
        # responses to P1:bet
        nodeUp  = 2
        nodeNew = 5
        self.gameTree[nodeUp]["conn_down"].append(nodeNew)
        self.gameTree[nodeNew] = {"name": "fold", 
                                  "thisPlayer": "p1", "nextPlayer": None, 
                                  "thisRound": thisRound, "nextRound": None,
                                  "conn_up": nodeUp, "conn_down": [],
                                  "winningPlayer": "p0",
                                  "winAmount": 1}
        self.playerActions["p1"].append(nodeNew)
        
        nodeNew = 6
        self.gameTree[nodeUp]["conn_down"].append(nodeNew)
        self.gameTree[nodeNew] = {"name": "call", 
                                  "thisPlayer": "p1", "nextPlayer": None, 
                                  "thisRound": thisRound, "nextRound": None,
                                  "conn_up": nodeUp, "conn_down": [],
                                  "winningPlayer": "SHOWDOWN",
                                  "winAmount": 2}
        self.playerActions["p1"].append(nodeNew)
     
    # ----------------------------------------------------------------------------------------------------
    #          third round
    # ----------------------------------------------------------------------------------------------------
    def addRound_3(self):
        thisRound, nextRound = 3, None
        
        # responses to P2:bet
        nodeUp  = 4
        nodeNew = 7
        self.gameTree[nodeUp]["conn_down"].append(nodeNew)
        self.gameTree[nodeNew] = {"name": "fold", 
                                  "thisPlayer": "p0", "nextPlayer": None, 
                                  "thisRound": thisRound, "nextRound": nextRound,
                                  "conn_up": nodeUp, "conn_down": [],
                                  "winningPlayer": "p1",
                                  "winAmount": 1}
        self.playerActions["p0"].append(nodeNew)
        
        nodeNew = 8
        self.gameTree[nodeUp]["conn_down"].append(nodeNew)
        self.gameTree[nodeNew] = {"name": "call", 
                                  "thisPlayer": "p0", "nextPlayer": None, 
                                  "thisRound": thisRound, "nextRound": nextRound,
                                  "conn_up": nodeUp, "conn_down": [],
                                  "winningPlayer": "SHOWDOWN",
                                  "winAmount": 2}
        self.playerActions["p0"].append(nodeNew)

# Function for traversing gameTree

In [27]:
###########################################################################################################
#         traverse the game tree
###########################################################################################################
def traverse_gameTree(gameTree, strategy, hand, hand_values):
    currentNode = 0
    gameOver = False

    dict_gamePlayed = {}

    while not gameOver:
        list_nextNodes = gameTree.gameTree[currentNode]["conn_down"]
        nextPlayer = gameTree.gameTree[currentNode]["nextPlayer"]
        thisRound = gameTree.gameTree[currentNode]["thisRound"]
        nextRound = gameTree.gameTree[currentNode]["nextRound"]

        # identify relevant strategy for player
        playerStrategy = strategy[nextPlayer]
        playerHand = hand[nextPlayer]

        # decide actions to take
        possible_actions = playerStrategy[playerHand][currentNode]

        arr_actions, arr_prob = np.array([]), np.array([]) 
        for action, prob in possible_actions.items():
            if action not in list_nextNodes:
                raise ValueError("One of the actions ({}) was not found in game-tree".format(action))

            arr_actions = np.append(arr_actions, action)
            arr_prob = np.append(arr_prob, prob)

        # pick action based on probability distribution
        chosen_action = int(np.random.choice(arr_actions, size=1, p=arr_prob))
        dict_gamePlayed[nextRound] = {"player": nextPlayer, "action": chosen_action}

        # move to next round
        currentNode = chosen_action

        # finish game if over
        if gameTree.gameTree[currentNode]["nextRound"] is None:
            gameOver = True

            # evaluate payoff
            winningPlayer = gt.gameTree[currentNode]["winningPlayer"]
            if winningPlayer == "SHOWDOWN":
                if hand_values[hand["p0"]] > hand_values[hand["p1"]]:
                    winningPlayer = "p0"
                elif hand_values[hand["p0"]] < hand_values[hand["p1"]]:
                    winningPlayer = "p1"
            winAmount = gameTree.gameTree[currentNode]["winAmount"]

    return winningPlayer, winAmount, dict_gamePlayed

# Remove weakly dominated strategies

https://en.wikipedia.org/wiki/Strategic_dominance <br>

**removing strictly dominated strategies** <br>
If we only remove strictly dominated strategy, the order of removal does not matter. 
We are also guaranteed to not remove any Nash equilibria from our solution.

**removing weakly dominated strategies** <br>
The order of removal matters, and we also might drop some Nash equilibria from our solution.
However, whatever remains after this process still is a valid Nash equilibrium.


In [94]:
###########################################################################################################
#         find weakly dominated strategies so that they can be removed
###########################################################################################################
def find_weaklyDominatedStrategies(mtx_payoff, findDominatedRows=False, findDominatedCols=False):
    
    if findDominatedRows and findDominatedCols:
        raise ValueError("Only one of [findDominatedRows / findDominatedCols] can be True")
        
    if not findDominatedRows and not findDominatedCols:
        raise ValueError("Only one of [findDominatedRows / findDominatedCols] can be False")
        
    if findDominatedRows:
        list_rowsDominated = []
        for iRow_toCheck in range(mtx_payoff.shape[0]):
            row_toCheck = mtx_payoff[iRow_toCheck] 

            isDominated = False

            for iRow in range(mtx_payoff.shape[0]):
                if iRow == iRow_toCheck:
                    continue

                thisRow = mtx_payoff[iRow]

                if all(row_toCheck <= thisRow):
                    isDominated = True
                    break

            if isDominated:
                list_rowsDominated.append(iRow_toCheck)
                
        return list_rowsDominated
                
    if findDominatedCols:
        list_colsDominated = []
        for iCol_toCheck in range(mtx_payoff.shape[1]):
            col_toCheck = mtx_payoff[:,iCol_toCheck] 

            isDominated = False

            for iCol in range(mtx_payoff.shape[1]):
                if iCol == iCol_toCheck:
                    continue

                thisCol = mtx_payoff[:,iCol]

                if all(col_toCheck <= thisCol):
                    isDominated = True
                    break

            if isDominated:
                list_colsDominated.append(iCol_toCheck)
                
        return list_colsDominated

In [145]:
###########################################################################################################
#         find weakly dominated strategies so that they can be removed
###########################################################################################################
def remove_weaklyDominatedStrategies(mtx_payoff, list_stratNames_p0, list_stratNames_p1):
    # ensure that we don't mess around with original arguments
    list_stratNames_p0 = copy.deepcopy(list_stratNames_p0)
    list_stratNames_p1 = copy.deepcopy(list_stratNames_p1)

    iteration = 0
    while True:
        iteration += 1
        print(f"[ITERATION {iteration}]")
        
        # [STEP 1] Find weakly dominated strategies (rows & cols)
        list_rowsDominated = find_weaklyDominatedStrategies(mtx_payoff, findDominatedRows=True)
        print("   Found {} weakly dominated strategies (rows)".format(
            len(list_rowsDominated)))

        # note that player1 (cols) has the same payoff as p0, but with reversed sign!
        list_colsDominated = find_weaklyDominatedStrategies(mtx_payoff*(-1), findDominatedCols=True)
        print("   Found {} weakly dominated strategies (cols)".format(
            len(list_colsDominated)))

        # [STEP 2] check if there are strategies to remove
        if (len(list_rowsDominated) == 0) and (len(list_colsDominated) == 0):
            print("Removal of weakly dominated strategies complete")
            return mtx_payoff, list_stratNames_p0, list_stratNames_p1
            
        # [STEP 3] delete all weakly dominated strategies from payoff_matrix
        mtx_payoff = np.delete(mtx_payoff, list_rowsDominated, axis=0)
        mtx_payoff = np.delete(mtx_payoff, list_colsDominated, axis=1)

        # [STEP 4] delete all weakly dominates strategies from strategyName list
        for idx_del in sorted(list_rowsDominated, reverse=True):
            del list_stratNames_p0[idx_del]

        for idx_del in sorted(list_colsDominated, reverse=True):
            del list_stratNames_p1[idx_del]

# Example - run 1 strategy with 1 pair of hands

In [4]:
gt = GameTree_Kuhn2P()

In [6]:
# strategy description
# strategy = {
#   "hand": $currentNode$: {action1: prob(action1), action2: prob(action2)}}

# 3 baseLine strategies for player-1 that can be applied to each hand
#  two actions are possible, depending on what p1 chose if p0 checks
#   --> strat_p0_CheckFold, strat_p0_CheckCall, strat_p0_Bet
strat_p0_CheckFold = {0: {1: 1.0, 2: 0.0} , 4: {7: 1.0, 8: 0.0}}
strat_p0_CheckCall = {0: {1: 1.0, 2: 0.0} , 4: {7: 0.0, 8: 1.0}}
strat_p0_Bet       = {0: {1: 0.0, 2: 1.0} , 4: None}

strategy_p0 = {
    "J": strat_p0_CheckFold,
    "Q": strat_p0_CheckCall,
    "K": strat_p0_Bet}

# 4 baseLine strategies for player-2 that can be applied to each hand
#  two actions are possible, depending on what p0 chose as first action (check or bet)
#  --> strat_p1_CheckFold, strat_p1_CheckCall, strat_p1_BetFold, strat_p1_BetCall
strat_p1_CheckFold = {1: {3: 1.0, 4: 0.0} , 2: {5: 1.0, 6: 0.0}}
strat_p1_CheckCall = {1: {3: 1.0, 4: 0.0} , 2: {5: 0.0, 6: 1.0}}
strat_p1_BetFold   = {1: {3: 0.0, 4: 1.0} , 2: {5: 1.0, 6: 0.0}}
strat_p1_BetCall   = {1: {3: 0.0, 4: 1.0} , 2: {5: 0.0, 6: 1.0}}

strategy_p1 = {
    "J": strat_p1_CheckFold,
    "Q": strat_p1_CheckCall,
    "K": strat_p1_BetCall}

strategy = {
    "p0": strategy_p0,
    "p1": strategy_p1}

In [37]:
# hand values:
hand_values = {"J": 1, "Q": 2, "K": 3}

# chose hands for players
hand = {"p0": "J", "p1": "Q"}

winningPlayer, winAmount, dict_gamePlayed = traverse_gameTree(
    gameTree=gt, 
    strategy=strategy, 
    hand=hand, 
    hand_values=hand_values)

print(f"{winningPlayer} won. winAmount={winAmount}")

p1 won. winAmount=1


In [111]:
dict_gamePlayed

{1: {'player': 'p0', 'action': 2}, 2: {'player': 'p1', 'action': 6}}

# Solve all strategies over all hands

In [19]:
list_hands = ["J", "Q", "K"]

#list_deals = ["JQ", "JK", "QJ", "QK", "KJ", "KQ"]
list_deals = [deal for deal in itertools.product(list_hands, list_hands)]

# remove duplicates
for hand in list_hands:
    list_deals.remove((hand, hand))

In [20]:
list_deals

[('J', 'Q'), ('J', 'K'), ('Q', 'J'), ('Q', 'K'), ('K', 'J'), ('K', 'Q')]

In [52]:
strat_p0_CheckFold = {0: {1: 1.0, 2: 0.0} , 4: {7: 1.0, 8: 0.0}}
strat_p0_CheckCall = {0: {1: 1.0, 2: 0.0} , 4: {7: 0.0, 8: 1.0}}
strat_p0_Bet       = {0: {1: 0.0, 2: 1.0} , 4: None}

strat_p1_CheckFold = {1: {3: 1.0, 4: 0.0} , 2: {5: 1.0, 6: 0.0}}
strat_p1_CheckCall = {1: {3: 1.0, 4: 0.0} , 2: {5: 0.0, 6: 1.0}}
strat_p1_BetFold   = {1: {3: 0.0, 4: 1.0} , 2: {5: 1.0, 6: 0.0}}
strat_p1_BetCall   = {1: {3: 0.0, 4: 1.0} , 2: {5: 0.0, 6: 1.0}}

strat_p0 = {
    "CheckFold": strat_p0_CheckFold,
    "CheckCall": strat_p0_CheckCall,
    "Bet"      : strat_p0_Bet}

strat_p1 = {
    "CheckFold": strat_p1_CheckFold,
    "CheckCall": strat_p1_CheckCall,
    "BetFold"  : strat_p1_BetFold,
    "BetCall"  : strat_p1_BetCall}

In [117]:
list_strategies_p0 = []
list_stratNames_p0 = []
for stratName_J, strat_J in strat_p0.items():
    for stratName_Q, strat_Q in strat_p0.items():
        for stratName_K, strat_K in strat_p0.items():
            strat = {"J": strat_J, "Q": strat_Q, "K": strat_K}
            list_strategies_p0.append(strat)
            
            stratNames = {"J": stratName_J, "Q": stratName_Q, "K": stratName_K}
            list_stratNames_p0.append(stratNames)
            
list_strategies_p1 = []
list_stratNames_p1 = []
for stratName_J, strat_J in strat_p1.items():
    for stratName_Q, strat_Q in strat_p1.items():
        for stratName_K, strat_K in strat_p1.items():
            strat = {"J": strat_J, "Q": strat_Q, "K": strat_K}
            list_strategies_p1.append(strat)
            
            stratNames = {"J": stratName_J, "Q": stratName_Q, "K": stratName_K}
            list_stratNames_p1.append(stratNames)

In [70]:
print("PLayer-1 strategies: ", len(list_strategies_p0))
print("PLayer-2 strategies: ", len(list_strategies_p1))

PLayer-1 strategies:  27
PLayer-2 strategies:  64


In [71]:
hand_values = {"J": 1, "Q": 2, "K": 3}

mtx_payoff = np.zeros((len(list_strategies_p0), len(list_strategies_p1)))

for iRow, p0_strategy in enumerate(list_strategies_p0):
    for iCol, p1_strategy in enumerate(list_strategies_p1):
        
        #print("Strategies:")
        #print("   p0: ", list_stratNames_p0[iRow])
        #print("   p1: ", list_stratNames_p1[iCol])
        
        payoff = 0.0
        for deal in list_deals:
            hand = {"p0": deal[0], "p1": deal[1]}
            
            # traverse gameTree to find payOff
            winningPlayer, winAmount, dict_gamePlayed = traverse_gameTree(
                gameTree=gt, 
                strategy={"p0": p0_strategy, "p1": p1_strategy}, 
                hand=hand, 
                hand_values=hand_values)
            
            if winningPlayer == "p0":
                payoff += winAmount
            elif winningPlayer == "p1":
                payoff -= winAmount
            else:
                raise RuntimeError(f"Unknown player {winningPlayer} won")
                
            #print( "     ", hand)
            #print(f"      {winningPlayer} won. winAmount={winAmount}")
            #print()
                
        mtx_payoff[iRow][iCol] = payoff

In [75]:
df_mtx = pd.DataFrame(mtx_payoff)
df_mtx.to_clipboard()

# Use iterated removal of weakly dominated strategies

In [146]:
mtx_payoff_clean, \
list_stratNames_p0_clean, \
list_stratNames_p1_clean = remove_weaklyDominatedStrategies(
    mtx_payoff, list_stratNames_p0, list_stratNames_p1)

[ITERATION 1]
   Found 15 weakly dominated strategies (rows)
   Found 56 weakly dominated strategies (cols)
[ITERATION 2]
   Found 4 weakly dominated strategies (rows)
   Found 4 weakly dominated strategies (cols)
[ITERATION 3]
   Found 0 weakly dominated strategies (rows)
   Found 0 weakly dominated strategies (cols)
Removal of weakly dominated strategies complete


In [148]:
mtx_payoff_clean.shape

(8, 4)

In [150]:
mtx_payoff_clean

array([[ 0.,  0., -1., -1.],
       [ 0.,  1., -2., -1.],
       [-1., -1.,  1.,  1.],
       [-1.,  0.,  0.,  1.],
       [ 1., -2.,  0., -3.],
       [ 1., -1., -1., -3.],
       [ 0., -3.,  2., -1.],
       [ 0., -2.,  1., -1.]])

In [149]:
list_stratNames_p0_clean

[{'J': 'CheckFold', 'Q': 'CheckFold', 'K': 'CheckCall'},
 {'J': 'CheckFold', 'Q': 'CheckFold', 'K': 'Bet'},
 {'J': 'CheckFold', 'Q': 'CheckCall', 'K': 'CheckCall'},
 {'J': 'CheckFold', 'Q': 'CheckCall', 'K': 'Bet'},
 {'J': 'Bet', 'Q': 'CheckFold', 'K': 'CheckCall'},
 {'J': 'Bet', 'Q': 'CheckFold', 'K': 'Bet'},
 {'J': 'Bet', 'Q': 'CheckCall', 'K': 'CheckCall'},
 {'J': 'Bet', 'Q': 'CheckCall', 'K': 'Bet'}]

# Scratch-pad, step-by-step

In [129]:
# find weakly dominated strategies
list_rowsDominated = find_weaklyDominatedStrategies(mtx_payoff, findDominatedRows=True)
print("Found {} weakly dominated strategies (rows)".format(len(list_rowsDominated)))

# note that player1 (cols) has the same payoff as p0, but with reversed sign!
list_colsDominated = find_weaklyDominatedStrategies(mtx_payoff*(-1), findDominatedCols=True)
print("Found {} weakly dominated strategies (cols)".format(len(list_colsDominated)))

Found 15 weakly dominated strategies (rows)
Found 56 weakly dominated strategies (cols)


In [130]:
# delete all weakly dominated strategies
mtx_payoff_cleaned = np.delete(mtx_payoff, list_rowsDominated, axis=0)
mtx_payoff_cleaned = np.delete(mtx_payoff_cleaned, list_colsDominated, axis=1)

list_stratNames_p0_cleaned = copy.deepcopy(list_stratNames_p0)
for idx_del in sorted(list_rowsDominated, reverse=True):
    del list_stratNames_p0_cleaned[idx_del]
    
list_stratNames_p1_cleaned = copy.deepcopy(list_stratNames_p1)
for idx_del in sorted(list_colsDominated, reverse=True):
    del list_stratNames_p1_cleaned[idx_del]

In [131]:
len(list_stratNames_p0_cleaned), len(list_stratNames_p1_cleaned)

(12, 8)

In [135]:
df_mtx_clean = pd.DataFrame(mtx_payoff_cleaned)
df_mtx_clean.to_clipboard()

In [134]:
# find weakly dominated strategies
list_rowsDominated2 = find_weaklyDominatedStrategies(mtx_payoff_cleaned, findDominatedRows=True)
print("Found {} weakly dominated strategies (rows)".format(len(list_rowsDominated2)))

# note that player1 (cols) has the same payoff as p0, but with reversed sign!
list_colsDominated2 = find_weaklyDominatedStrategies(mtx_payoff_cleaned*(-1), findDominatedCols=True)
print("Found {} weakly dominated strategies (cols)".format(len(list_colsDominated2)))

Found 4 weakly dominated strategies (rows)
Found 4 weakly dominated strategies (cols)


In [138]:
# delete all weakly dominated strategies
mtx_payoff_cleaned2 = np.delete(mtx_payoff_cleaned, list_rowsDominated2, axis=0)
mtx_payoff_cleaned2 = np.delete(mtx_payoff_cleaned2, list_colsDominated2, axis=1)

list_stratNames_p0_cleaned2 = copy.deepcopy(list_stratNames_p0_cleaned)
for idx_del in sorted(list_rowsDominated2, reverse=True):
    del list_stratNames_p0_cleaned2[idx_del]
    
list_stratNames_p1_cleaned2 = copy.deepcopy(list_stratNames_p1_cleaned)
for idx_del in sorted(list_colsDominated2, reverse=True):
    del list_stratNames_p1_cleaned2[idx_del]

In [139]:
mtx_payoff_cleaned2.shape

(8, 4)

In [140]:
# find weakly dominated strategies
list_rowsDominated3 = find_weaklyDominatedStrategies(mtx_payoff_cleaned2, findDominatedRows=True)
print("Found {} weakly dominated strategies (rows)".format(len(list_rowsDominated3)))

# note that player1 (cols) has the same payoff as p0, but with reversed sign!
list_colsDominated3 = find_weaklyDominatedStrategies(mtx_payoff_cleaned2*(-1), findDominatedCols=True)
print("Found {} weakly dominated strategies (cols)".format(len(list_colsDominated3)))

Found 0 weakly dominated strategies (rows)
Found 0 weakly dominated strategies (cols)


In [141]:
list_stratNames_p0_cleaned2

[{'J': 'CheckFold', 'Q': 'CheckFold', 'K': 'CheckCall'},
 {'J': 'CheckFold', 'Q': 'CheckFold', 'K': 'Bet'},
 {'J': 'CheckFold', 'Q': 'CheckCall', 'K': 'CheckCall'},
 {'J': 'CheckFold', 'Q': 'CheckCall', 'K': 'Bet'},
 {'J': 'Bet', 'Q': 'CheckFold', 'K': 'CheckCall'},
 {'J': 'Bet', 'Q': 'CheckFold', 'K': 'Bet'},
 {'J': 'Bet', 'Q': 'CheckCall', 'K': 'CheckCall'},
 {'J': 'Bet', 'Q': 'CheckCall', 'K': 'Bet'}]

In [142]:
list_stratNames_p1_cleaned2

[{'J': 'CheckFold', 'Q': 'CheckFold', 'K': 'BetCall'},
 {'J': 'CheckFold', 'Q': 'CheckCall', 'K': 'BetCall'},
 {'J': 'BetFold', 'Q': 'CheckFold', 'K': 'BetCall'},
 {'J': 'BetFold', 'Q': 'CheckCall', 'K': 'BetCall'}]